# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
weather_file = "temp.csv"
weather_df = pd.read_csv(weather_file)
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Husavik,40,IS,1584295777,74,66.04,-17.34,30.20,42.50
1,1,Bambous Virieux,75,MU,1584295777,83,-20.34,57.76,78.80,1.12
2,2,Caculé,100,BR,1584295778,68,-14.50,-42.22,79.14,6.15
3,3,Hobart,90,AU,1584295778,71,-42.88,147.33,53.60,4.65
4,4,Tuktoyaktuk,90,CA,1584295778,77,69.45,-133.04,-2.20,21.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
locations= weather_df[["Lat", "Lng"]]
humidity= weather_df["Humidity"].astype(float)


In [59]:
gmaps.configure(api_key=g_key)


In [60]:
fig= gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig

###How do I get this to show?


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
max_temp= weather_df["Max Temp"].astype(float)


max_temp

0      30.20
1      78.80
2      79.14
3      53.60
4      -2.20
5      66.20
6      77.88
7      51.01
8      62.60
9      66.90
10     75.20
11     83.28
12     91.38
13     52.00
14     89.40
15     89.60
16     78.78
17     -6.72
18     84.20
19     78.80
20     17.60
21     64.99
22     85.26
23     68.00
24     33.80
25     60.01
26     55.40
27     54.00
28     86.00
29     84.20
       ...  
532    86.00
533    18.84
534    75.36
535    74.64
536    -3.12
537    82.99
538    12.00
539    38.10
540    71.60
541    80.60
542    39.20
543    75.72
544    26.60
545    25.32
546    77.00
547    37.99
548    37.99
549    44.60
550    26.60
551    71.60
552    84.20
553    30.20
554    44.04
555    23.00
556    28.92
557    69.60
558    78.96
559    77.00
560    81.48
561    59.00
Name: Max Temp, Length: 562, dtype: float64

In [83]:
hotel_df=  weather_df[(weather_df["Max Temp"]>75) & (weather_df["Max Temp"]<80) & (weather_df["Cloudiness"]==0) & (weather_df["Wind Speed"]<10)]

hotel_df["Hotel Name"]= ""
hotel_df

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
45,45,Coron,0,PH,1584295781,76,12.00,120.20,77.88,1.48,
163,163,Aswān,0,EG,1584295789,33,24.09,32.91,77.00,6.93,
234,234,Mount Isa,0,AU,1584295794,31,-20.73,139.50,75.20,8.05,
455,455,Viedma,0,AR,1584295810,18,-40.81,-63.00,76.98,2.46,
525,525,Nhulunbuy,0,AU,1584295815,88,-12.23,136.77,77.00,7.05,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [99]:
# geocoordinates
hotel_name=[]
###how do I get this correctly
lat=hotel_df["Lat"]
lng=hotel_df["Lng"]
for i, row in hotel_df.iterrows():
    lat, lng = row['Lat'], row['Lng']
    target_coordinates = f"{lat},{lng}"
    target_radius = 5000
    target_type= "lodging"
    #print(target_coordinates)
# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "key": g_key,
        "type": target_type
        }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    places_data = response.json()
    #print(places_data)

    try:
         hotel_name1=places_data["results"][0]["name"]
         print(hotel_name1)
         hotel_name.append(hotel_name1)
         
    except: 
        print("Hotel not found")
    
    #print(places_data["results"][0]["vicinity"])
hotel_df["Hotel Name"]= hotel_name

Coron Westown Resort
Nuba Nile
ibis Styles Mt Isa Verona
Hotel Cristal
East Arnhem Real Estate


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [100]:
places_data

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -12.196515, 'lng': 136.7648868},
    'viewport': {'northeast': {'lat': -12.1950548197085,
      'lng': 136.7662045802915},
     'southwest': {'lat': -12.1977527802915, 'lng': 136.7635066197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
   'id': 'eb144a2e76d7b38a53a793ebb440eb07409dfce3',
   'name': 'East Arnhem Real Estate',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 1365,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109355539661370775075">East Arnhem Real Estate</a>'],
     'photo_reference': 'CmRaAAAAkmkj-YvjVXtb7ZFgYw7plczCzOjIwAB4bmiK6Wmi_fnV5wojwmqAoAElk4UPyHaX3vdvq32bTEQ0B97BzReVYyDuAxRSEkLsNk-XJtdi3qg4LlYCB0r2b-5nLlLef5xBEhBDgXVckvAHxZT9t0Ft0V9lGhTAiWWEHy-tbqEk7A6-nsvGq-MW3g',
     'width': 2048}],
   'place_id': 'ChIJvRS436KA3WkRwi-0ZNU_Sww',
   'plus_code': {'compound_code': 'RQ37+9X Nhulunbuy, Northern Ter

In [101]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
45,45,Coron,0,PH,1584295781,76,12.00,120.20,77.88,1.48,Coron Westown Resort
163,163,Aswān,0,EG,1584295789,33,24.09,32.91,77.00,6.93,Nuba Nile
234,234,Mount Isa,0,AU,1584295794,31,-20.73,139.50,75.20,8.05,ibis Styles Mt Isa Verona
455,455,Viedma,0,AR,1584295810,18,-40.81,-63.00,76.98,2.46,Hotel Cristal
525,525,Nhulunbuy,0,AU,1584295815,88,-12.23,136.77,77.00,7.05,East Arnhem Real Estate


In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_hotel = hotel_df[["Lat", "Lng"]]

In [93]:
###How do I add the infobox?
locations_weather= weather_df[["Lat", "Lng"]]
fig = gmaps.figure(layout=figure_layout)
heat_layer=gmaps.heatmap_layer(locations_weather, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)


# Add marker layer ontop of heat map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#info box content
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations_hotel, info_box_content= hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…